In [ ]:
import numpy as np
import mpmath
from pyFDTR.domain import *
from pyFDTR.materials import *
from pyFDTR.fouriermodel import *
import matplotlib as mpl
import matplotlib.pyplot as plt


mpmath.mp.dps = 5

load = np.genfromtxt('./sapphire-gold60nm_80K.txt',skip_header=2)
points80k = np.delete(load,1,1)

load = np.genfromtxt('./sapphire-gold60nm_300K.txt',skip_header=2)
points300k = np.delete(load,1,1)

#frequencies = points300k[:,0]
frequencies = [1e3,5e3,20e3,50e3,200e3,500e3,1e6,3e6,6e6,10e6,15e6,20e6,40e6]

domain = Domain(80)
domain.add_substrate(sapphire)
domain.add_layer(60e-7,gold)
domain.set_interface_condu(1,5e3)
model = FourierModelFDTR(domain,4.05e-4,4.05e-4,0)

print(vars(domain.heat_path[0]))
print(vars(domain.heat_path[1]))
print(vars(domain.heat_path[2]))

phases_80 = []
for f in frequencies:
  phases_80.append(model.get_phase(f))

domain = Domain(300)
domain.add_substrate(sapphire)
domain.add_layer(60e-7,gold)
domain.set_interface_condu(1,5e3)
#domain.set_layer_param(1,kt=33.6,kp=33.6)
model = FourierModelFDTR(domain,4.05e-4,4.05e-4,0)

print(vars(domain.heat_path[0]))
print(vars(domain.heat_path[1]))
print(vars(domain.heat_path[2]))

phases_300 = []
for f in frequencies:
  phases_300.append(model.get_phase(f))

load = np.genfromtxt('./sapphire-gold60nm_80K.txt',skip_header=2)
points80k = np.delete(load,1,1)

load = np.genfromtxt('./sapphire-gold60nm_300K.txt',skip_header=2)
points300k = np.delete(load,1,1)

fig, ax = plt.subplots()  # Create a figure containing a single axes.
ax.semilogx(frequencies, phases_80);  # Plot some data on the axes.
ax.semilogx(frequencies, phases_300);  # Plot some data on the axes.
ax.scatter(points80k[:,0],points80k[:,1])
ax.scatter(points300k[:,0],points300k[:,1])